# Armed Banking: The Final Notebook

 _Helena Rodríguez Gómez_                                     
02/12/2022

***
This notebook documents my Data Journalism Project.
It has 5 main sections:
1. Motivation of the project (what I want to answer with my analysis) 
2. Data wrangling: Cleaning and exploring Dataset 1
3. Data wrangling: Cleaning and exploring Datasets 2 and 3
4. Data aggregation & some first analyses
5. Data visualisation & final analyses

But first of all, I import the libraries and modules I am going to use.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Motivation of the project (what I want to answer with my analysis)

I want to find out which banking institutions finance arms companies, and which countries they come from. I am especially interested in banking institutions and arm companies based on the **European Union**.
Relatedly, I want to see where these arms end up, and explore the arms flows.
This is relevant for:
* Accountability--> Looking into where our money ends up
* Uncommon angle to the story: going beyond national defense budgets. 
* Worldwide increase in military spending 
* Widespread bellicist discourses in the public, mediatic and political debate.

## 2. Data wrangling: Cleaning and exploring Dataset 1

#### About the dataset
The Delas Worldwide Armed Banks Database is is the first dataset I will use in my story and transform it into an object. 
I found the first dataset in the [Centre Delàs de Cultura per la Pau website](https://www.sipri.org/databases/armstransfers). But it had a problem: it did not allow me to download the information regarding the country origin of the banking institutions.


Then Freddy came to the rescue and build a scraper that allows us to retrieve the excel files for every country and then renames the file to the selected country.We then loop over all files, take the country name from the file and insert it as a new column to the data, and then build one large dataframe. 


I attach the code that Frederick used in a **separate .ipynb file**, in case the reader wants to execute the code of the Delàs web scrapper.

Before loading it, a few notes on what this dataset contains and where it comes from.

This dataset comes from the Delàs Center for Culture of Peace, dedicated to research on arms investment and production, and military spending. It is designed to present data of the financing of arms companies made by financial institutions. 

Their participation in the arms business is done through one or more types of financing, broken down into 4 categories: 
* the issuance of bonds and promissory notes; 
* lending to arms companies; 
* investment funds, stocks and bonds in arms companies and 
* the management of pension funds. 

ATTENTION: It was **last updated in September 2020, with aggregated data from 2014 to 2019**, and a lot has changed in the world regarding armed conflicts and military spending since then. This could be a shortcoming of the dataset if we aim to produce a news story, since newness is one of the key news values. 

The limited geographic span is another shortcoming to keep in mind: it only includes information of banking entities **from 56 countries**, and it **does not include Russian banks and arming companies**, which would be a considerable limitation considering the current war in Europe. Nonetheless, it does include substantial information of financial institutions from the US (764 entities), Spain (42) or France (33) among others, of special interest for a European audience.

Let's upload our data using the pandas library and store it in a dataframe.

In [2]:
pd.read_csv('bank_country.csv')
df_bank_country = pd.read_csv('bank_country.csv')
df_bank_country

,Unnamed: 0,Financial institution,Arms company,Loans to arms companies,"Issuance of bonds, shares and promissory notes of armament companies",Ownership or management of share and bonds of arms companies,Total by financial institution,country
0,0,Vanguard,Saab,-,-,-,4.709716e+07,Sweden
1,1,Royal Bank of Scotland,Saab,-,-,-,6.194447e+07,Sweden
2,2,Danske Bank,Saab,-,-,-,2.978019e+08,Sweden
3,3,Lannebo Fonder,Saab,-,-,-,6.619983e+07,Sweden
4,4,AFA Försäkring,Saab,-,-,-,1.879882e+08,Sweden
...,...,...,...,...,...,...,...,...
4345,13,Fidelity International (Bermuda),Rolls-Royce,-,-,"41.708.751,45$",4.170875e+07,Bermuda
4346,14,Lazard (Bermuda),Rolls-Royce,-,-,"30.441.749,56$",3.044175e+07,Bermuda
4347,15,Orbis Group (Bermuda),Rolls-Royce,-,-,"134.378.928,21$",1.343789e+08,Bermuda
4348,16,White Mountains Insurance (Bermuda),Rolls-Royce,-,-,1.000.000$,1.000000e+06,Bermuda


NOTE: While in the website you can also see the financial tool category "Pension funds", I could not find it in the downloaded sheet... so I won't include this financial tool in my analyses.

We can also see that the first column is an index, so I will get rid of it with `.drop` since pandas already creates one. The names of the columns are quite long and that can be unpractical to operate with, so I will `.rename` them as well. I will also `.replace` the dashes for a standardized missing value indication, **NaN**. Finally, I will store all these cleaning changes into a new, clean dataframe called **clean_delas**. 

In [3]:
clean_delas=df_bank_country.drop(columns=['Unnamed: 0'])
clean_delas=clean_delas.rename(columns={"Financial institution":"fin_inst","Arms company":"arms_comp","Loans to arms companies":"loans","Issuance of bonds, shares and promissory notes of armament companies":"issuance","Ownership or management of share and bonds of arms companies":"ownership", "Total by financial institution":"total_by_inst"})
clean_delas= clean_delas.replace('-', np.nan)
clean_delas

,fin_inst,arms_comp,loans,issuance,ownership,total_by_inst,country
0,Vanguard,Saab,NaN,NaN,NaN,4.709716e+07,Sweden
1,Royal Bank of Scotland,Saab,NaN,NaN,NaN,6.194447e+07,Sweden
2,Danske Bank,Saab,NaN,NaN,NaN,2.978019e+08,Sweden
3,Lannebo Fonder,Saab,NaN,NaN,NaN,6.619983e+07,Sweden
4,AFA Försäkring,Saab,NaN,NaN,NaN,1.879882e+08,Sweden
...,...,...,...,...,...,...,...
4345,Fidelity International (Bermuda),Rolls-Royce,NaN,NaN,"41.708.751,45$",4.170875e+07,Bermuda
4346,Lazard (Bermuda),Rolls-Royce,NaN,NaN,"30.441.749,56$",3.044175e+07,Bermuda
4347,Orbis Group (Bermuda),Rolls-Royce,NaN,NaN,"134.378.928,21$",1.343789e+08,Bermuda
4348,White Mountains Insurance (Bermuda),Rolls-Royce,NaN,NaN,1.000.000$,1.000000e+06,Bermuda


Now I want to perform some **first descriptive statistics** to get to know the distribution of the data better, because most of it is hidden when we try to see it. First, I try the `.describe` method for the total dataset, and it returns the descriptive stats of the total_by_inst.
Still, it's not giving me very useful info to answer my RQ:

In [4]:
clean_delas.describe()

,total_by_inst
count,4.350000e+03
mean,2.610817e+08
std,1.166851e+09
min,1.426100e+02
25%,4.500836e+06
50%,3.428509e+07
75%,1.399430e+08
max,3.148664e+10


I want to check how many distinct bank entities, arms companies and countries are in the dataset. I'll use `.describe` but for each of the columns for that.

In [5]:
clean_delas["country"].describe()

count                         4350
unique                          41
top       United States of America
freq                          2191
Name: country, dtype: object

Strange... even though they say there are 55 countries, we can only see 41 different ones... Let's see which ones picking only the country column.

In [6]:
countries = clean_delas[["country"]].groupby("country").nunique()
countries

""
country
Andorra
Australia
Austria
Bahamas
Bahrein
Belgium
Bermuda
Brazil
Bulgaria


In [7]:
clean_delas["fin_inst"].describe()

count         4350
unique        1225
top       Vanguard
freq            49
Name: fin_inst, dtype: object

In [8]:
clean_delas["arms_comp"].describe()

count                       4350
unique                        39
top       Raytheon  technologies
freq                         980
Name: arms_comp, dtype: object

This gives us an interesting first overview of the dataset: 
* 41 countries, at the top, the US. Out of the 41, 15 are from the European Union (this means that 12 of the 27 EU countries are missing). There is also a "European Union" entry.
* 1225 financial institutions, at the top, Vanguard
* Only 39 different arms companies are included in the dataset; at the top, Raytheon technologies

### General Top 10s

**I want to find out the top 10 financial institutions** and see where they're from.
To do so, we need to use `.groupby` and `.sort_values` in a descending way, and create a new dataframe (that I'll name **top10**) containing the *fin_inst*, the *total_by_inst* and the *country* columns, and finish with `.head(10)`

Moreover, I'll reset the index so that it looks like a top 10, using `reset_index(drop=True)` and defining the range from 1 to 11 (taking into consideration Python's 0 indexing).

In [9]:
#DOUBT: How can I add here the column country?
top10 = clean_delas.groupby("fin_inst").sum()["total_by_inst"].sort_values(ascending=False).head(10)
top10 = top10.reset_index()
top10.index = range(1, 11)
top10

,fin_inst,total_by_inst
1,Capital Group,8.688434e+10
2,BlackRock,8.203301e+10
3,Vanguard,7.700081e+10
4,State Street,5.811103e+10
5,T. Rowe Price,3.675792e+10
6,Bank of America,3.526261e+10
7,JPMorgan Chase,3.162944e+10
8,Verisight,3.150871e+10
9,Citigroup,2.524890e+10
10,Mitsubishi UFJ Financial,2.394942e+10


I want to explore my dataset per countries to see where is most of the money coming from.

In [10]:
delas_total_per_countries= clean_delas.groupby("country").sum()["total_by_inst"].sort_values(ascending=False)
pd.DataFrame(delas_total_per_countries)

,total_by_inst
country,
United States of America,7.860969e+11
France,1.154046e+11
Netherlands,8.118058e+10
Japan,2.926197e+10
Canada,2.738987e+10
Germany,1.839130e+10
Spain,1.646292e+10
Italy,1.130356e+10
Taiwan,7.016439e+09


Now I'll get the top10 countries

In [11]:
top10_countries_inst= delas_total_per_countries.sort_values(ascending=False).head(10)
top10_countries_inst = top10_countries_inst.reset_index()
top10_countries_inst.index = range(1, 11)
pd.DataFrame(top10_countries_inst)

,country,total_by_inst
1,United States of America,7.860969e+11
2,France,1.154046e+11
3,Netherlands,8.118058e+10
4,Japan,2.926197e+10
5,Canada,2.738987e+10
6,Germany,1.839130e+10
7,Spain,1.646292e+10
8,Italy,1.130356e+10
9,Taiwan,7.016439e+09
10,China,6.740269e+09


To get a better picture of what these numbers represent, I will **plot them into a bar chart in section 5.**

Additional finding: The **top 10 financial institutions** that invest through different investment types are **all from the USA** (I found out Googling them). Nonetheless, It is important to remember that this dataset does not include information from Russian institutions.

***Note:*** Maybe it would be meaningful as well to see the proportion that the top 10 fin. institutions represent out of the total? I am going to try that by getting the total sum by financial institution, and adding all the billions that the top 10 represent and comparing them with the rest.

In [12]:
totalsum_top10fininst = top10['total_by_inst'].sum()
totalsum_top10fininst

488386194518.22003

In [13]:
totalsum_fin_inst = clean_delas["total_by_inst"].sum()
totalsum_fin_inst

1135705179863.72

In [14]:
difference_fin_inst = totalsum_fin_inst - totalsum_top10fininst
difference_fin_inst

647318985345.5

In [15]:
10/1243

0.008045052292839904

In [16]:
difference_fin_inst/totalsum_fin_inst

0.5699709720644012

So, out of the 1243 financial insitutions, **the top 10 of them (that is, the 0.8% of the total) invested almost half of the total money given to arms companies between 2014 and 2019**. I would like to visualise this.

Now, the top **receiving arms companies**:

In [17]:
top10armscompanies = clean_delas.groupby("arms_comp").sum()["total_by_inst"].sort_values(ascending=False).head(10)
top10armscompanies = top10armscompanies.reset_index()
top10armscompanies.index = range(1, 11)
top10armscompanies

,arms_comp,total_by_inst
1,Boeing,2.866120e+11
2,Honeywell International,9.231513e+10
3,General Dynamics,8.788709e+10
4,Lockheed Martin,8.634509e+10
5,Airbus,8.168658e+10
6,Northrop Grumman Innovation Systems,6.065902e+10
7,Raytheon technologies,5.772519e+10
8,Rolls-Royce,4.611660e+10
9,Safran,4.278296e+10
10,BAE Systems,4.039146e+10


I am going to compare how much these receive compared to the rest.

In [18]:
totalsum_top10armscomp = top10armscompanies['total_by_inst'].sum()
totalsum_top10armscomp

882521158493.0101

In [19]:
difference_arms_comp = totalsum_fin_inst - totalsum_top10armscomp
difference_arms_comp

253184021370.70984

In [20]:
totalsum_top10armscomp/totalsum_fin_inst

0.7770688856054254

In [21]:
10/39

0.2564102564102564

I visualise this in the 5th section

### Slicing and subsetting: zooming into the EU
I want to select and see only the EU financial institutions. To do so, I will use one of the subsetting methods that Freddy showed us in class, that allows us to get only the rows in which our column of interest is identical to the string ‘xx’ that we select.

First I'll look into separate countries. I am going to explore the EU countries whose banking institutions are investing the most (remembering that France, Germany, Spain and Italy are at the top 10).

In [22]:
France_delas_dataset = clean_delas[clean_delas['country']=='France']
France_delas_dataset

,fin_inst,arms_comp,loans,issuance,ownership,total_by_inst,country
46,Crédit Agricole,Aecom,NaN,NaN,NaN,8.193019e+08,France
47,Crédit Mutuel CIC Group,Aecom,NaN,NaN,NaN,4.358369e+07,France
48,BNP Paribas,Aecom,NaN,NaN,NaN,1.699743e+09,France
49,AXA,Aecom,NaN,NaN,NaN,1.327400e+07,France
50,BPCE Group,Aerojet Rocketdyne,NaN,NaN,NaN,2.867461e+07,France
...,...,...,...,...,...,...,...
385,AG2R La Mondiale,Rolls-Royce,NaN,NaN,"15.277.701,08$",1.527770e+07,France
386,Carmignac Gestion,Rolls-Royce,NaN,NaN,"177.007.697,87$",1.770077e+08,France
387,Groupama,Rolls-Royce,NaN,NaN,5.663.000$,5.663000e+06,France
388,BNP Paribas,Rolls-Royce,"374.437.556,35$",262.900.000$,10.438.000$,6.477756e+08,France


In [23]:
France_delas_dataset.describe()

,total_by_inst
count,3.440000e+02
mean,3.354785e+08
std,1.040947e+09
min,1.162924e+04
25%,8.134500e+06
50%,7.500000e+07
75%,3.025291e+08
max,1.518259e+10


In [24]:
France_delas_dataset["fin_inst"].describe()

count             344
unique            172
top       BNP Paribas
freq               23
Name: fin_inst, dtype: object

In [25]:
France_delas_dataset["arms_comp"].describe()

count          344
unique          30
top       Dassault
freq            95
Name: arms_comp, dtype: object

In [26]:
top10FinInstFR = France_delas_dataset.groupby("fin_inst").sum()["total_by_inst"].sort_values(ascending=False).head(10)
top10FinInstFR = top10FinInstFR.reset_index()
top10FinInstFR.index = range(1, 11)
top10FinInstFR

,fin_inst,total_by_inst
1,Capital Group,1.971386e+10
2,BNP Paribas,1.240376e+10
3,Crédit Agricole,9.882968e+09
4,Société Générale,8.964022e+09
5,BlackRock,8.171212e+09
6,Dassault Family,6.772489e+09
7,BPCE Group,4.582753e+09
8,Sociedad Estatal de Participaciones Industriales,3.219036e+09
9,Vanguard,3.045144e+09
10,Deutsche Bank,2.555825e+09


In [27]:
Germany_delas_dataset = clean_delas[clean_delas['country']=='Germany']
Germany_delas_dataset

,fin_inst,arms_comp,loans,issuance,ownership,total_by_inst,country
2592,Munich Re,Aecom,NaN,NaN,NaN,4.300000e+07,Germany
2593,Commerzbank,Aecom,NaN,NaN,NaN,1.005247e+08,Germany
2594,Allianz,Aecom,NaN,NaN,NaN,1.361570e+08,Germany
2595,Deutsche Bank,Aerojet Rocketdyne,NaN,NaN,NaN,2.888784e+07,Germany
2596,Allianz,Aerojet Rocketdyne,NaN,NaN,NaN,2.500000e+06,Germany
...,...,...,...,...,...,...,...
2709,LOYS,Rolls-Royce,NaN,NaN,"4.141.382,79$",4.141383e+06,Germany
2710,Landesbank Hessen-Thüringen,Rolls-Royce,NaN,NaN,1.110.000$,1.110000e+06,Germany
2711,DZ Bank,Rolls-Royce,NaN,NaN,"13.691.556,16$",1.369156e+07,Germany
2712,Deutsche Bank,Rolls-Royce,"132.767.254,22$",NaN,"39.625.953,9$",1.723932e+08,Germany


In [28]:
Netherlands_delas_dataset = clean_delas[clean_delas['country']=='Netherlands']


In [29]:
Top10Netherlands_finist = Netherlands_delas_dataset.groupby('fin_inst').sum()['total_by_inst'].sort_values(ascending=False).head(10)
Top10Netherlands_finist = Top10Netherlands_finist.reset_index()
Top10Netherlands_finist.index = range(1, 11)
Top10Netherlands_finist

,fin_inst,total_by_inst
1,Janus Henderson,6.421778e+09
2,Barclays,5.900664e+09
3,Lloyds Banking Group,4.940976e+09
4,Invesco,3.909438e+09
5,HSBC,3.778540e+09
6,Capital Group,3.425755e+09
7,Legal & General,2.772896e+09
8,Royal Bank of Scotland,2.490838e+09
9,BlackRock,2.239056e+09
10,ValueAct Capital Management,2.124017e+09


Most of these financial institutions are NOT registered in the Netherlands... 

In [30]:
Spain_delas_dataset = clean_delas[clean_delas['country']=='Spain']
Spain_delas_dataset

,fin_inst,arms_comp,loans,issuance,ownership,total_by_inst,country
2874,Santander *Incluye Banco Popular,Safran,NaN,NaN,NaN,4.875614e+08,Spain
2875,Banco Bilbao Vizcaya Argentaria (BBVA),Aecom,NaN,NaN,NaN,5.269344e+08,Spain
2876,Banco de Sabadell,Aecom,NaN,NaN,NaN,7.358504e+07,Spain
2877,Santander *Incluye Banco Popular,Serco,NaN,NaN,NaN,1.430987e+07,Spain
2878,Magallanes,Serco,NaN,NaN,NaN,1.674679e+07,Spain
...,...,...,...,...,...,...,...
3042,Banco Bilbao Vizcaya Argentaria (BBVA),Navantia,"87.148.870,94$",NaN,NaN,8.714887e+07,Spain
3043,Santander *Incluye Banco Popular,Navantia,"38.733.434,47$",NaN,NaN,3.873343e+07,Spain
3044,Lloyds Banking Group,Navantia,"85.780.301,89$",NaN,NaN,8.578030e+07,Spain
3045,Crédit Agricole,Navantia,"58.099.609,06$",NaN,NaN,5.809961e+07,Spain


In [31]:
Spain_delas_dataset.describe()

,total_by_inst
count,1.730000e+02
mean,9.516139e+07
std,3.253578e+08
min,4.196710e+03
25%,1.110000e+06
50%,6.822083e+06
75%,7.173888e+07
max,3.219036e+09


In [32]:
Spain_delas_dataset["fin_inst"].describe()

count                                  173
unique                                 115
top       Santander *Incluye Banco Popular
freq                                    15
Name: fin_inst, dtype: object

In [33]:
Spain_delas_dataset["arms_comp"].describe()

count        173
unique        21
top       Indra 
freq          94
Name: arms_comp, dtype: object

In [34]:
Spain_delas_dataset["arms_comp"].unique()

array(['Safran', 'Aecom', 'Serco', 'Airbus', 'Aselsan ', 'Thales',
       'Leonardo ', 'Dassault', 'Raytheon  technologies', 'Rolls-Royce',
       'Boeing', 'BWX Technologies', 'Elbit Systems', 'Embraer', 'Fluor',
       'General Dynamics', 'Honeywell International ', 'Indra ',
       'Jacobs Engineering', 'Maxam ', 'Navantia'], dtype=object)

In [35]:
Spaintop10fin_inst = Spain_delas_dataset.groupby("fin_inst").sum()["total_by_inst"].sort_values(ascending=False).head(10)
Spaintop10fin_inst

fin_inst
Banco Bilbao Vizcaya Argentaria (BBVA)              5.011294e+09
Sociedad Estatal de Participaciones Industriales    3.596219e+09
Santander *Incluye Banco Popular                    3.294736e+09
BNP Paribas                                         4.238376e+08
Citigroup                                           3.684724e+08
Banca March (March Group)                           2.428200e+08
Barclays                                            2.176162e+08
BFA Holding                                         2.174633e+08
HSBC                                                2.072987e+08
Crédit Agricole                                     2.004597e+08
Name: total_by_inst, dtype: float64

In [36]:
Germany_delas_dataset["fin_inst"].describe()


count               122
unique               54
top       Deutsche Bank
freq                 19
Name: fin_inst, dtype: object

In [37]:
Germany_delas_dataset["arms_comp"].describe()

count                122
unique                26
top       Rheinmetall AG
freq                  35
Name: arms_comp, dtype: object

In [38]:
Top10Germany_finist = Germany_delas_dataset.groupby('fin_inst').sum()['total_by_inst'].sort_values(ascending=False).head(10)
Top10Germany_finist = Top10Germany_finist.reset_index()
Top10Germany_finist.index = range(1, 11)
Top10Germany_finist

,fin_inst,total_by_inst
1,Deutsche Bank,8.578249e+09
2,Commerzbank,1.988681e+09
3,Allianz,1.738717e+09
4,DZ Bank,1.557129e+09
5,BayernLB,9.258617e+08
6,Capital Group,5.914944e+08
7,European Investment Bank,2.874723e+08
8,BlackRock,2.331607e+08
9,Deka Group,1.679649e+08
10,IKB Deutsche Industriebank Aktiengesellschaft,1.631612e+08


In [39]:
Italy_delas_dataset = clean_delas[clean_delas['country']=='Italy']
Top10Italy_finist = Italy_delas_dataset.groupby('fin_inst').sum()['total_by_inst'].sort_values(ascending=False).head(10)
Top10Italy_finist = Top10Italy_finist.reset_index()
Top10Italy_finist.index = range(1, 11)
Top10Italy_finist

,fin_inst,total_by_inst
1,UniCredit,2.472690e+09
2,Intesa Sanpaolo,1.062038e+09
3,BNP Paribas,8.877226e+08
4,Cassa Depositi e Prestiti,7.335563e+08
5,Crédit Agricole,3.739760e+08
6,European Investment Bank,3.417900e+08
7,Norwegian Government Pension Fund - Global,2.492685e+08
8,Citigroup,2.343616e+08
9,JPMorgan Chase,2.335278e+08
10,BayernLB,2.265600e+08


Now I'll try to do the same but with all the EU countries.

In [40]:
EU_delas_dataset = clean_delas[(clean_delas['country']=='Austria')|(clean_delas['country']=='Belgium')| (clean_delas['country']=='Bulgaria')| (clean_delas['country']=='Denmark')| (clean_delas['country']=='Finland')| (clean_delas['country']=='France')|(clean_delas['country']=='Germany') | (clean_delas['country']=='Ireland') | (clean_delas['country']=='Italy') | (clean_delas['country']=='Luxembourg')| (clean_delas['country']=='Netherlands')| (clean_delas['country']=='Poland')| (clean_delas['country']=='Portugal')| (clean_delas['country']=='Spain')| (clean_delas['country']=='Sweden') | (clean_delas['country']=='European Union')]
EU_delas_dataset

,fin_inst,arms_comp,loans,issuance,ownership,total_by_inst,country
0,Vanguard,Saab,NaN,NaN,NaN,4.709716e+07,Sweden
1,Royal Bank of Scotland,Saab,NaN,NaN,NaN,6.194447e+07,Sweden
2,Danske Bank,Saab,NaN,NaN,NaN,2.978019e+08,Sweden
3,Lannebo Fonder,Saab,NaN,NaN,NaN,6.619983e+07,Sweden
4,AFA Försäkring,Saab,NaN,NaN,NaN,1.879882e+08,Sweden
...,...,...,...,...,...,...,...
4327,Evli Bank,Saab,NaN,NaN,NaN,4.105598e+07,Finland
4328,Sampo Group,Raytheon technologies,NaN,NaN,"4.821.565,44$",4.821565e+06,Finland
4329,OP Financial Group,Raytheon technologies,NaN,NaN,"3.587.647,42$",3.587647e+06,Finland
4330,Säästöpankki,Rolls-Royce,NaN,NaN,"8.324.449,85$",8.324450e+06,Finland


In [41]:
EU_delas_dataset.describe()

,total_by_inst
count,1.465000e+03
mean,1.713073e+08
std,5.761917e+08
min,4.196710e+03
25%,4.186371e+06
50%,3.333224e+07
75%,1.294230e+08
max,1.518259e+10


In [42]:
total_investment_EU_banks= EU_delas_dataset["total_by_inst"].sum()

In [43]:
print ("This is the total amount of dollars that banks based on the EU (included in this dataset) invest in arms companies:")
total_investment_EU_banks

This is the total amount of dollars that banks based on the EU (included in this dataset) invest in arms companies:


250965162636.16998

In [44]:
#It would be interesting to compare with the total amount of banks based on the rest of the world
# and to compare it with banks based on the US as well. ANd maybe with China?

Now I want to know the top10 financial institutions based in the EU that finance arms companies

In [45]:
EUtop10 = EU_delas_dataset.groupby("fin_inst").sum()["total_by_inst"].sort_values(ascending=False).head(10)
EUtop10 = EUtop10.reset_index()
EUtop10.index = range(1, 11)
EUtop10

,fin_inst,total_by_inst
1,Capital Group,2.373111e+10
2,BNP Paribas,1.456717e+10
3,Deutsche Bank,1.180727e+10
4,Crédit Agricole,1.150692e+10
5,BlackRock,1.082745e+10
6,Société Générale,9.690693e+09
7,Janus Henderson,6.920246e+09
8,Barclays,6.899788e+09
9,Sociedad Estatal de Participaciones Industriales,6.815255e+09
10,Dassault Family,6.772489e+09


#### Important note
The country category seems to be confusing and not coherent across the dataset, since some companies from the US appear in the European database, as well as some german in the french one, etc. Therefore, we need to look at it with caution.

In [46]:
#Freddys code to try to add the country
EUtop10.merge(EU_delas_dataset, on=['fin_inst'])

,fin_inst,total_by_inst_x,arms_comp,loans,issuance,ownership,total_by_inst_y,country
0,Capital Group,2.373111e+10,Airbus,NaN,NaN,NaN,1.518259e+10,France
1,Capital Group,2.373111e+10,Safran,NaN,NaN,NaN,4.220148e+09,France
2,Capital Group,2.373111e+10,Thales,NaN,NaN,NaN,3.111168e+08,France
3,Capital Group,2.373111e+10,Rheinmetall AG,NaN,NaN,NaN,5.914944e+08,Germany
4,Capital Group,2.373111e+10,BAE Systems,NaN,NaN,NaN,2.237001e+09,Netherlands
...,...,...,...,...,...,...,...,...
163,Barclays,6.899788e+09,Leonardo,111.150.000$,"61.401.818,18$",NaN,1.725518e+08,Italy
164,Sociedad Estatal de Participaciones Industriales,6.815255e+09,Airbus,NaN,NaN,NaN,3.219036e+09,France
165,Sociedad Estatal de Participaciones Industriales,6.815255e+09,Airbus,NaN,NaN,NaN,3.219036e+09,Spain
166,Sociedad Estatal de Participaciones Industriales,6.815255e+09,Indra,NaN,NaN,"377.182.876,74$",3.771829e+08,Spain


In [47]:
#I am trying to also display the country but it doesn't work
#EUtop10_country = EU_delas_dataset.groupby("fin_inst").sum()["total_by_inst"].sort_values(ascending=False).head(10).concat(EU_delas_dataset[["country"]])
#EUtop10_country 


I want to see the total per country of the EU

In [48]:
EU_total_per_countries= EU_delas_dataset.groupby("country").sum()["total_by_inst"].sort_values(ascending=False)
pd.DataFrame(EU_total_per_countries)


,total_by_inst
country,
France,1.154046e+11
Netherlands,8.118058e+10
Germany,1.839130e+10
Spain,1.646292e+10
Italy,1.130356e+10
Sweden,5.881066e+09
European Union,1.295830e+09
Denmark,6.425742e+08
Austria,9.196650e+07


Now I will go on to the next datasets; and I will visualise this information in sections 4 and 5.

## 3. Data wrangling: Cleaning and exploring Datasets 2 and 3


First, a few notes on what these two SIPRI Arms Transfers Databases contain. They feature statistical data on actual deliveries of major conventional weapons. 

[As the authors explain](https://www.sipri.org/databases/armstransfers/sources-and-methods), SIPRI has developed a unique system to measure the volume of international transfers of major conventional weapons using a common unit, the **trend-indicator value (TIV)**, with the goal of permitting comparison and identify general trends: 

* The TIV is based on the known unit production costs of a core set of weapons and is **intended to represent the transfer of military resources rather than the financial value of the transfer**. 

* SIPRI TIV figures **do not represent sales prices for arms transfers**. They should therefore not be directly compared with gross domestic product (GDP), military expenditure, sales values or the financial value of export licences in an attempt to measure the economic burden of arms imports or the economic benefits of exports. 

* They are **best used as the raw data for calculating trends in international arms transfers over periods of time, global percentages for suppliers and recipients, and percentages for the volume of transfers to or from particular states.**

### 3.1 First, I will look at the importing countries.

I upload and clean the dataset, and then look at the top 10.

In [49]:
df_allimports = pd.read_csv("Clean_TIV-Import-All-2020-2021.csv")
df_allimports.rename(columns = {'Unnamed: 0':'country'}, inplace = True)
df_allimports

,country,2020,2021,Total
0,Afghanistan,240.0,69.0,310
1,Albania,1.0,NaN,1
2,Algeria,661.0,237.0,898
3,Angola,51.0,NaN,51
4,Argentina,7.0,77.0,84
...,...,...,...,...
133,Vanuatu,NaN,7.0,7
134,Venezuela,79.0,NaN,79
135,Viet Nam,32.0,244.0,275
136,Zambia,1.0,19.0,20


To be able to compare the different countries and not distort the data with the total count (which is in row 137), i will create a new dataframe without the total and I will call it "clean_allimports". I will also focus only on one year, the most recent one.

In [50]:
#clean_allimports=df_allimports.drop([137], axis=0, inplace=True)
clean_allimports=df_allimports.iloc[:137]
clean_allimports2021=clean_allimports.drop(columns=['2020','Total'])
clean_allimports2021

,country,2021
0,Afghanistan,69.0
1,Albania,NaN
2,Algeria,237.0
3,Angola,NaN
4,Argentina,77.0
...,...,...
132,Uzbekistan,47.0
133,Vanuatu,7.0
134,Venezuela,NaN
135,Viet Nam,244.0


In [51]:
clean_allimports2021.country

0      Afghanistan
1          Albania
2          Algeria
3           Angola
4        Argentina
          ...     
132     Uzbekistan
133        Vanuatu
134      Venezuela
135       Viet Nam
136         Zambia
Name: country, Length: 137, dtype: object

I want to see the **top importing countries in 2021.**

In [52]:
top10importers2021 = clean_allimports2021['2021'].sort_values(ascending=False).head(10)
top10importers2021 = top10importers2021.reset_index()
top10importers2021.index = range(1, 11)
top10importers2021

,index,2021
1,52,4414.0
2,100,1767.0
3,103,1723.0
4,38,1355.0
5,6,1235.0
6,28,901.0
7,66,899.0
8,128,893.0
9,60,885.0
10,91,884.0


In [53]:
#i am trying to display the name of the country here
top10importers2021 = clean_allimports2021.sort_values("2021", ascending=False)[["country", "2021"]].head(10)
top10importers2021= top10importers2021.reset_index()
top10importers2021.index = range(1, 11)
top10importers2021clean= top10importers2021.drop(columns=['index'])
top10importers2021clean

,country,2021
1,India,4414.0
2,Qatar,1767.0
3,Saudi Arabia,1723.0
4,Egypt,1355.0
5,Australia,1235.0
6,China,901.0
7,Kuwait,899.0
8,United Kingdom,893.0
9,Japan,885.0
10,Pakistan,884.0


Again, I want to see what the top 10 mean in respect of the rest.

In [54]:
total_top10importers = top10importers2021clean['2021'].sum()
total_top10importers

14956.0

In [55]:
total_all2021imports = clean_allimports2021['2021'].sum()
total_all2021imports

25640.0

In [56]:
difference_topimp_andrest = total_all2021imports - total_top10importers
difference_topimp_andrest

10684.0

In [57]:
total_top10importers/total_all2021imports

0.5833073322932917

### 3.2 Now, I will look at my third dataset, concerning exporting countries.

I will follow the same cleaning and analaytical steps.

In [58]:
df_allexports = pd.read_csv("Clean_TIV-Export-All-2020-2021.csv")
df_allexports.rename(columns = {'Unnamed: 0':'country'}, inplace = True)
clean_allexports=df_allexports.iloc[:45]
clean_allexports2021=clean_allexports.drop(columns=['2020','Total'])
clean_allexports2021

,country,2021
0,Australia,173.0
1,Austria,16.0
2,Belarus,98.0
3,Belgium,48.0
4,Brazil,88.0
5,Bulgaria,3.0
6,Canada,119.0
7,China,1085.0
8,Colombia,NaN
9,Czechia,1.0


In [59]:
top10exporters2021 = clean_allexports2021.sort_values("2021", ascending=False)[["country", "2021"]].head(10)
top10exporters2021 = top10exporters2021.reset_index()
top10exporters2021.index = range(1, 11)
top10exporters2021clean= top10exporters2021.drop(columns=['index'])
top10exporters2021clean


,country,2021
1,United States,10613.0
2,France,3954.0
3,Russia,2744.0
4,Italy,1717.0
5,China,1085.0
6,Germany,914.0
7,Spain,612.0
8,Israel,606.0
9,United Kingdom,601.0
10,South Korea,566.0


### 3.3 Zooming in into EU

In [60]:
#I will directly use the merged imports and exports database
#EU_im_exports2021left = im_exports2021left[(im_exports2021left['country']=='Austria')|(im_exports2021left['country']=='Belgium')| (im_exports2021left['country']=='Bulgaria')| (im_exports2021left['country']=='Croatia')| (im_exports2021left['country']=='Republic of Cyprus')| (im_exports2021left['country']=='Czech Republic')| (im_exports2021left['country']=='Denmark')| (im_exports2021left['country']=='Estonia')| (im_exports2021left['country']=='Finland')| (im_exports2021left['country']=='France')|(im_exports2021left['country']=='Germany') | Greece   Hungary, (im_exports2021left['country']=='Ireland') | (clean_delas['country']=='Italy') | Latvia, Lithuania (clean_delas['country']=='Luxembourg')| Malta (clean_delas['country']=='Netherlands')| (clean_delas['country']=='Poland')| (clean_delas['country']=='Portugal')| Romania, Slovakia, Slovenia (clean_delas['country']=='Spain')| (clean_delas['country']=='Sweden')]
#EU_im_exports2021left


## 4. Data aggregation & some first analyses


Initially, I intended to merge delas with sipri datasets, but since they refer to different time periods, they use different units of measures and the delas is featuring way less countries than the sipri ones, I decided to merge the sipri's imports and the exports dataframes into one dataset to visualise it together.

### 4.1 Merging and getting the balance of imports and exports

In [61]:
im_exports2021left = pd.merge(clean_allimports2021, clean_allexports2021,how='left', on='country')
im_exports2021left = im_exports2021left.rename(columns = {'2021_x':'imports'})
im_exports2021left =im_exports2021left.rename(columns = {'2021_y':'exports'})
im_exports2021left



,country,imports,exports
0,Afghanistan,69.0,NaN
1,Albania,NaN,NaN
2,Algeria,237.0,NaN
3,Angola,NaN,NaN
4,Argentina,77.0,NaN
...,...,...,...
132,Uzbekistan,47.0,NaN
133,Vanuatu,7.0,NaN
134,Venezuela,NaN,NaN
135,Viet Nam,244.0,NaN


Now I will get the difference between imports and exports to get the arms trade balance per country.

In [62]:
im_exports2021left_with_balance = im_exports2021left.assign(balance=lambda im_exports2021left: im_exports2021left.imports-im_exports2021left.exports)
im_exports2021left_with_balance 

,country,imports,exports,balance
0,Afghanistan,69.0,NaN,NaN
1,Albania,NaN,NaN,NaN
2,Algeria,237.0,NaN,NaN
3,Angola,NaN,NaN,NaN
4,Argentina,77.0,NaN,NaN
...,...,...,...,...
132,Uzbekistan,47.0,NaN,NaN
133,Vanuatu,7.0,NaN,NaN
134,Venezuela,NaN,NaN,NaN
135,Viet Nam,244.0,NaN,NaN


I want to get rid of the NaN, so that we get **only coutries that import and export**, and with full data.

In [63]:
clean_im_exports2021left_with_balance = im_exports2021left_with_balance.dropna()
clean_im_exports2021left_with_balance.reset_index()
clean_im_exports2021left_with_balance = clean_im_exports2021left_with_balance.sort_values("balance", ascending=False)[["country", "imports","exports","balance"]]
clean_im_exports2021left_with_balance = clean_im_exports2021left_with_balance.reset_index()
clean_im_exports2021left_with_balance.index = range(1, 39)
clean_im_exports2021left_with_balance= clean_im_exports2021left_with_balance.drop(columns=['index'])
clean_im_exports2021left_with_balance

,country,imports,exports,balance
1,India,4414.0,33.0,4381.0
2,Qatar,1767.0,29.0,1738.0
3,Australia,1235.0,173.0,1062.0
4,Pakistan,884.0,53.0,831.0
5,Norway,492.0,58.0,434.0
6,UAE,440.0,48.0,392.0
7,Denmark,345.0,27.0,318.0
8,Indonesia,328.0,17.0,311.0
9,United Kingdom,893.0,601.0,292.0
10,Netherlands,589.0,299.0,290.0


In [64]:
clean_im_exports2021left_with_balance.balance

1     4381.0
2     1738.0
3     1062.0
4      831.0
5      434.0
6      392.0
7      318.0
8      311.0
9      292.0
10     290.0
11     260.0
12     207.0
13     154.0
14     146.0
15     135.0
16     135.0
17     133.0
18     125.0
19     106.0
20      99.0
21      52.0
22      29.0
23      22.0
24      18.0
25       4.0
26      -2.0
27      -4.0
28     -36.0
29     -47.0
30    -109.0
31    -137.0
32    -184.0
33    -187.0
34    -487.0
35    -755.0
36   -1488.0
37   -3908.0
38   -9875.0
Name: balance, dtype: float64

### 4.2 Zooming in into EU with balance of imports and exports

Now I'll make one with only EU countries (of imports and exports).

In [65]:
#EU_clean_2021_TIVbalance = clean_im_exports2021left_with_balance[(clean_im_exports2021left_with_balance['country'] == '')|(clean_im_exports2021left_with_balance['Country'] == '')|(clean_im_exports2021left_with_balance['Country'] == '')|(clean_im_exports2021left_with_balance['Country'] == '')|(clean_im_exports2021left_with_balance['Country'] == '')|(clean_im_exports2021left_with_balance['Country'] == '')|(clean_im_exports2021left_with_balance['Country'] == '')|(clean_im_exports2021left_with_balance['Country'] == '')|(clean_im_exports2021left_with_balance['Country'] == '')|(clean_im_exports2021left_with_balance['Country'] == '')|(clean_im_exports2021left_with_balance['Country'] == '')|(clean_im_exports2021left_with_balance['Country'] == '')|(clean_im_exports2021left_with_balance['Country'] == '')|(clean_im_exports2021left_with_balance['Country'] == '')|(clean_im_exports2021left_with_balance['Country'] == '')|(clean_im_exports2021left_with_balance['Country'] == '')|(clean_im_exports2021left_with_balance['Country'] == '')|(clean_im_exports2021left_with_balance['Country'] == '')|
#THIS CODE IS UNFINISHED, FINISH DOING THIS PART
#EU_delas_dataset = clean_delas[(clean_delas['country']=='Austria')|(clean_delas['country']=='Belgium')| (clean_delas['country']=='Bulgaria')| (clean_delas['country']=='Denmark')| (clean_delas['country']=='Finland')| (clean_delas['country']=='France')|(clean_delas['country']=='Germany') | (clean_delas['country']=='Ireland') | (clean_delas['country']=='Italy') | (clean_delas['country']=='Luxembourg')| (clean_delas['country']=='Netherlands')| (clean_delas['country']=='Poland')| (clean_delas['country']=='Portugal')| (clean_delas['country']=='Spain')| (clean_delas['country']=='Sweden') | (clean_delas['country']=='European Union')]
#EU_delas_dataset

### 4.3 Crossing top countries that finance with top countries that export


Now I will see whether there is a correlation between the countries that most finance arms companies and the ones that most export arms.

In [66]:
fin_and_exports = pd.merge(delas_total_per_countries, clean_allexports2021,how='left', on='country')
fin_and_exports = fin_and_exports.rename(columns = {'2021':'arms_exports'})
fin_and_exports = fin_and_exports.rename(columns = {'total_by_inst':'total_by_financial_inst'})
fin_and_exports


,country,total_by_financial_inst,arms_exports
0,United States of America,7.860969e+11,NaN
1,France,1.154046e+11,3954.0
2,Netherlands,8.118058e+10,299.0
3,Japan,2.926197e+10,NaN
4,Canada,2.738987e+10,119.0
5,Germany,1.839130e+10,914.0
6,Spain,1.646292e+10,612.0
7,Italy,1.130356e+10,1717.0
8,Taiwan,7.016439e+09,3.0
9,China,6.740269e+09,1085.0


In [67]:
corr_matrix = fin_and_exports.corr()
corr_matrix

,total_by_financial_inst,arms_exports
total_by_financial_inst,1.000000,0.738817
arms_exports,0.738817,1.000000


This 0.74 value would indicate a high correlation between countries that invest most in arms companies and countries that export arms more, which would make sense.

#### Trying out more stuff

Now I'll try to explore the overlap between the two datasets through a different way. I am going to try to see whether there is a correlation between the top 20 financing and the top 20 exporting using `.concat`.



In [68]:
top20exporters2021 = clean_allexports2021.sort_values("2021", ascending=False)[["country", "2021"]].head(20)
top20exporters2021 = top20exporters2021.reset_index()
top20exporters2021.index = range(1, 21)
top20exporters2021clean= top20exporters2021.drop(columns=['index'])
top20exporters2021clean

,country,2021
1,United States,10613.0
2,France,3954.0
3,Russia,2744.0
4,Italy,1717.0
5,China,1085.0
6,Germany,914.0
7,Spain,612.0
8,Israel,606.0
9,United Kingdom,601.0
10,South Korea,566.0


In [69]:
top20_countries_inst= delas_total_per_countries.sort_values(ascending=False).head(20)
top20_countries_inst = top20_countries_inst.reset_index()
top20_countries_inst.index = range(1, 21)
pd.DataFrame(top20_countries_inst)

,country,total_by_inst
1,United States of America,7.860969e+11
2,France,1.154046e+11
3,Netherlands,8.118058e+10
4,Japan,2.926197e+10
5,Canada,2.738987e+10
6,Germany,1.839130e+10
7,Spain,1.646292e+10
8,Italy,1.130356e+10
9,Taiwan,7.016439e+09
10,China,6.740269e+09


In [70]:
comparison_top20s = pd.concat([top20_countries_inst, top20exporters2021], axis=1)
comparison_top20s = comparison_top20s.rename(columns = {'total_by_inst':'Total by financial institutions'})
comparison_top20s = comparison_top20s.rename(columns = {'2021':'Arms exports in year 2021'})
comparison_top20s = comparison_top20s.rename(columns = {'country': 'Country'})
comparison_top20s.drop(columns=['index'])

,Country,Total by financial institutions,Country,Arms exports in year 2021
1,United States of America,7.860969e+11,United States,10613.0
2,France,1.154046e+11,France,3954.0
3,Netherlands,8.118058e+10,Russia,2744.0
4,Japan,2.926197e+10,Italy,1717.0
5,Canada,2.738987e+10,China,1085.0
6,Germany,1.839130e+10,Germany,914.0
7,Spain,1.646292e+10,Spain,612.0
8,Italy,1.130356e+10,Israel,606.0
9,Taiwan,7.016439e+09,United Kingdom,601.0
10,China,6.740269e+09,South Korea,566.0


## 5. Data visualisation & final analyses


### 5.1 Visualising the top10s

In [71]:
!pip install pygal
import pygal
import bokeh
%matplotlib inline

**Top10 Financial Institutions, globally**

In [72]:
bar_chart_Top10FinInst = pygal.Bar()
bar_chart_Top10FinInst.title = 'Top 10 Financial Institutions financing arms companies in the world (in Billions USD)'
bar_chart_Top10FinInst.add('Capital Group', 86.88434)
bar_chart_Top10FinInst.add('BlackRock', 82.03301)
bar_chart_Top10FinInst.add('Vanguard', 77.00081)
bar_chart_Top10FinInst.add('State Street', 58.11103)
bar_chart_Top10FinInst.add('T. Rowe Price', 36.75792)
bar_chart_Top10FinInst.add('Bank of America', 35.26261)
bar_chart_Top10FinInst.add('JPMorgan Chase', 31.62944)
bar_chart_Top10FinInst.add('Verisight', 31.50871)
bar_chart_Top10FinInst.add('Citigroup', 25.2489)
bar_chart_Top10FinInst.add('Mitsubishi UFJ Financial', 23.94942)  
#bar_chart_Top10FinInst.render_in_browser()
#bar_chart_Top10FinInst.render_to_file('/tmp/chart.svg') 
bar_chart_Top10FinInst.render_to_png('top10chart.png')
# /mysvg.svg

Visualizing the volume of the top 10 financial institutions in relation to the total with a **pie chart**. 

Let's try do do an interactive one with bokeh. I will convert the data to billions.

In [73]:
top10

,fin_inst,total_by_inst
1,Capital Group,8.688434e+10
2,BlackRock,8.203301e+10
3,Vanguard,7.700081e+10
4,State Street,5.811103e+10
5,T. Rowe Price,3.675792e+10
6,Bank of America,3.526261e+10
7,JPMorgan Chase,3.162944e+10
8,Verisight,3.150871e+10
9,Citigroup,2.524890e+10
10,Mitsubishi UFJ Financial,2.394942e+10


In [74]:
difference_fin_inst = totalsum_fin_inst - totalsum_top10fininst
difference_fin_inst

647318985345.5

In [75]:
from math import pi

from bokeh.palettes import Category20c
from bokeh.plotting import figure, show
from bokeh.transform import cumsum

x = {
    'Capital Group': 86.884340000,
    'BlackRock': 82.033010000,
    'Vanguard': 77.000810000,
    'State Street': 58.111030000,
    'T. Rowe Price': 36.757920000,
    'Bank of America': 35.262610000,
    'JPMorgan Chase': 31.629440000,
    'Verisight': 31.508710000,
    'Citigroup': 25.248900000,
    'Mitsubishi UFJ Financial': 23.949420000,
    'Rest of financial institutions': 647.318985345
}

data = pd.Series(x).reset_index(name='value').rename(columns={'index': 'Financial institution'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = Category20c[len(x)]

pie_difference_fin_inst = figure(height=350, title="The top 10 banking institution financing arms producers, and the rest", toolbar_location=None,
           tools="hover", tooltips="@value", x_range=(-0.5, 1.0))

pie_difference_fin_inst.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='Financial institution', source=data)

pie_difference_fin_inst.axis.axis_label = None
pie_difference_fin_inst.axis.visible = False
pie_difference_fin_inst.grid.grid_line_color = None

# save to file (How????)
#p=output_to_file
#output_to_file('top10_and_therest.html')
#p.render_to_file("top10_and_therest.svg")
#pie_difference_fin_inst.render_to_png('pie difference.png')
show(pie_difference_fin_inst)

#trying to get an embeddable html

#from bokeh.resources import CDN
#from bokeh.embed import file_html

#plot = figure()
#plot.circle([1,2], [3,4])

#html = file_html(plot, CDN, "my plot")

**Top10 arms companies, globally**

In [76]:
top10armscompanies

,arms_comp,total_by_inst
1,Boeing,2.866120e+11
2,Honeywell International,9.231513e+10
3,General Dynamics,8.788709e+10
4,Lockheed Martin,8.634509e+10
5,Airbus,8.168658e+10
6,Northrop Grumman Innovation Systems,6.065902e+10
7,Raytheon technologies,5.772519e+10
8,Rolls-Royce,4.611660e+10
9,Safran,4.278296e+10
10,BAE Systems,4.039146e+10


Now I will visualize their volume respect the total.

In [77]:
totalsum_top10armscomp

882521158493.0101

In [78]:
difference_arms_comp

253184021370.70984

In [79]:
top10armscompanies

,arms_comp,total_by_inst
1,Boeing,2.866120e+11
2,Honeywell International,9.231513e+10
3,General Dynamics,8.788709e+10
4,Lockheed Martin,8.634509e+10
5,Airbus,8.168658e+10
6,Northrop Grumman Innovation Systems,6.065902e+10
7,Raytheon technologies,5.772519e+10
8,Rolls-Royce,4.611660e+10
9,Safran,4.278296e+10
10,BAE Systems,4.039146e+10


In [80]:
from math import pi

from bokeh.palettes import Category20c
from bokeh.plotting import figure, show
from bokeh.transform import cumsum

x = {
    'Boeing': 286.612000000,
    'Honeywell International': 92.315130000,
    'General Dynamics' : 87.887090000,
    'Lockheed Martin' : 86.345090000,
    'Airbus': 81.6865800000,
    'Northrop Grumman Innovation Systems':60.659020000,
    'Raytheon technologies':57.725190000,
    'Rolls-Royce': 46.116600000,
    'Safran':42.782960000,
    'BAE Systems':40.391460000,
    'Rest of arms companies': 253.18402137070984
}

data = pd.Series(x).reset_index(name='value').rename(columns={'index': 'Arms company'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = Category20c[len(x)]

pie_difference_arms_comp = figure(height=350, title="The top 10 receiving arms companies and the rest", toolbar_location=None,
           tools="hover", tooltips="@value", x_range=(-0.5, 1.0))

pie_difference_arms_comp.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='Arms company', source=data)

pie_difference_arms_comp.axis.axis_label = None
pie_difference_arms_comp.axis.visible = False
pie_difference_arms_comp.grid.grid_line_color = None

show(pie_difference_arms_comp)



#### Top 10 financial inst. by countries

In [81]:
top10_countries_inst

,country,total_by_inst
1,United States of America,7.860969e+11
2,France,1.154046e+11
3,Netherlands,8.118058e+10
4,Japan,2.926197e+10
5,Canada,2.738987e+10
6,Germany,1.839130e+10
7,Spain,1.646292e+10
8,Italy,1.130356e+10
9,Taiwan,7.016439e+09
10,China,6.740269e+09


#### Top 10 Financial institutions in the EU 

In [82]:
EUtop10

,fin_inst,total_by_inst
1,Capital Group,2.373111e+10
2,BNP Paribas,1.456717e+10
3,Deutsche Bank,1.180727e+10
4,Crédit Agricole,1.150692e+10
5,BlackRock,1.082745e+10
6,Société Générale,9.690693e+09
7,Janus Henderson,6.920246e+09
8,Barclays,6.899788e+09
9,Sociedad Estatal de Participaciones Industriales,6.815255e+09
10,Dassault Family,6.772489e+09


In [83]:
# HERE BAR CHART??? ANOTHER TYPE OF CHART??? MMAYBE A CHART THAT HAS BOTH GLOBAL AND EU IN ONE VISUALISATION??????
#Janud Henderson, Barclays = UK
#

I wanted to compare what european banks invest vs. what banks from the rest invest, but I have the uncertainty that some of the top banks retrieved from eu are actually originally from the US, but the quantity they invest is way smaller than the global one.

In [84]:
totalsum_nonEUbanks= totalsum_fin_inst - total_investment_EU_banks
totalsum_nonEUbanks

884740017227.55

In [85]:
from math import pi

from bokeh.palettes import Category20c
from bokeh.plotting import figure, show
from bokeh.transform import cumsum
from bokeh.plotting import figure, output_file, save

from bokeh.io import export_png
from bokeh.io import output_notebook
from bokeh.plotting import figure


x = {
    'Capital Group': 23.73111,
    'BNP Paribas': 14.56717,
  'Deutsche Bank': 11.80727,
'Crédit Agricole' : 11.50692,
'BlackRock' :  10.82745,
'Société Générale': 9.690693,
'Janus Henderson': 6.920246,
'Barclays': 6.899788,
'Sociedad Estatal de Participaciones Industriales':6.815255,
'Dassault Family' : 6.772489,
'Total sum of banks out of the EU': 884.74001722755 
}
data = pd.Series(x).reset_index(name='value').rename(columns={'index': 'Financial institutions in the EU'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = Category20c[len(x)]

piechart_EUfinance_andrest = figure(height=350, title="The top 10 Financial institutions in the EU and the rest (values in billions USD)", toolbar_location=None,
           tools="hover", tooltips="@value", x_range=(-0.5, 1.0))

piechart_EUfinance_andrest.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='Financial institutions in the EU', source=data)

piechart_EUfinance_andrest.axis.axis_label = None
piechart_EUfinance_andrest.axis.visible = False
piechart_EUfinance_andrest.grid.grid_line_color = None


# save the results to a file
export_png(piechart_EUfinance_andrest, filename="piechart_EUfinance_andrest.png")
#see the result in browser
show(piechart_EUfinance_andrest)


In [86]:
#trying to get an html file
from bokeh.plotting import figure
from bokeh.resources import CDN
from bokeh.embed import file_html

piechart_EUfinance_andrest = figure()
html = file_html(piechart_EUfinance_andrest, CDN, "my plot")

In [87]:
EU_total_per_countries

country
France            1.154046e+11
Netherlands       8.118058e+10
Germany           1.839130e+10
Spain             1.646292e+10
Italy             1.130356e+10
Sweden            5.881066e+09
European Union    1.295830e+09
Denmark           6.425742e+08
Austria           9.196650e+07
Luxembourg        7.064664e+07
Belgium           6.758109e+07
Finland           5.943032e+07
Ireland           4.613121e+07
Portugal          4.358369e+07
Poland            1.707858e+07
Bulgaria          6.330000e+06
Name: total_by_inst, dtype: float64

### 5.2 Visualising imports and exports

##### MAPS
First, I am going to try to display some of my data in maps, with a specific pygal extension I found here:
https://www.pygal.org/en/stable/documentation/types/maps/pygal_maps_world.html


In [88]:
pip install pygal_maps_world

Note: you may need to restart the kernel to use updated packages.


I can plot countries by specifying their code (specified in their documentation; problem: not all countries supported). ***Big problem!! It does not support Qatar, 2nd biggest importer in terms of TIV balance...***

In [89]:
top20exporters2021

,index,country,2021
1,43,United States,10613.0
2,12,France,3954.0
3,29,Russia,2744.0
4,19,Italy,1717.0
5,7,China,1085.0
6,13,Germany,914.0
7,34,Spain,612.0
8,18,Israel,606.0
9,42,United Kingdom,601.0
10,33,South Korea,566.0


In [90]:
top20importers2021 = clean_allimports2021.sort_values("2021", ascending=False)[["country", "2021"]].head(20)
top20importers2021

,country,2021
52,India,4414.0
100,Qatar,1767.0
103,Saudi Arabia,1723.0
38,Egypt,1355.0
6,Australia,1235.0
28,China,901.0
66,Kuwait,899.0
128,United Kingdom,893.0
60,Japan,885.0
91,Pakistan,884.0


In [91]:
worldmap_chart_importing = pygal.maps.world.World()
worldmap_chart_importing.title = 'Top 20 Importing countries'
worldmap_chart_importing.add('Top 20', {'in': 1, 'sa':3, 'au':5,'eg':4,'cn':6,'pk':10, 'kw':7,'no':15,'ae':16,'dk':19, 'jp':9, 'us':11, 'id':20,
                                           'gb':8,'nl':14,'kr':12, 'tm':17, 'ph':18,'il':13})


# Render the map to a file
worldmap_chart_importing.render_to_file('worldmap_top20importers.svg')

In [92]:
worldmap_chart_importing.render_in_browser()

file://C:/Users/Helena/AppData/Local/Temp/tmpgmmef0bm.html


In [94]:
#I'm gonna try to change the color.
from pygal.style import RotateStyle
dark_rotate_style = RotateStyle('#9e6ffe')
worldmap_chart_exporting = pygal.maps.world.World(style=dark_rotate_style)
worldmap_chart_exporting.title = 'Top 20 Exporting countries'
worldmap_chart_exporting.add('Top 20', {'by':17,'ua':19,'tr':11,'se':12,'ch':15,'cn':5,'nl':13,'ru':3,'il':8,'gb':9, 
                                'kr':10, 'au':14, 'br':18,'es':7,'de':6,'it':4,'fr':2,'us':1,'za':20,'ca':16}, tooltip= 'Top 20 Export')

# Render the map to a file
worldmap_chart_exporting.render_to_file('worldmap_top20exporters.svg')

In [95]:
worldmap_chart_exporting.render_in_browser()

file://C:/Users/Helena/AppData/Local/Temp/tmptlzp4y1t.html


Now I'll compare and visualize the top 10 importers with the rest

In [ ]:
top10importers2021clean

In [ ]:
total_top10importers = top10importers2021clean['2021'].sum()
total_top10importers

In [ ]:
difference_topimp_andrest

In [ ]:
x = {
    'India':4414.0,
    'Qatar':1767.0,
    'Saudi Arabia': 1723.0,
    'Egypt':1355.0,
    'Australia':1235.0,
    'China'	:901.0,
    'Kuwait': 899.0,
    'United Kingdom': 893.0,
    'Japan': 885.0,
    'Pakistan': 884.0,
    'The rest of importing countries': 10684.0
}

data = pd.Series(x).reset_index(name='value').rename(columns={'index': 'Importing countries'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = Category20c[len(x)]

pie_difference_importing_countries = figure(height=350, title="The top 10 importing countries in 2021 and the rest", toolbar_location=None,
           tools="hover", tooltips="@value", x_range=(-0.5, 1.0))

pie_difference_importing_countries.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='Importing countries', source=data)

pie_difference_importing_countries.axis.axis_label = None
pie_difference_importing_countries.axis.visible = False
pie_difference_importing_countries.grid.grid_line_color = None

# save the results to a file
export_png(pie_difference_importing_countries, filename="pie_difference_importing_countries.png")
#see the result in browser
show(pie_difference_importing_countries)


I'm going to visualise the differences between imports and exports. To do so, **I need to create a specific variable and save it in its own column  called import/export; in long-format instead of wide**. To do so, I use `pd.melt`

In [ ]:
im_exports2021_wide= pd.melt(im_exports2021left, id_vars='country', value_vars= ['imports', 'exports'], var_name='type_of_exchange', value_name='Trend_Indicator_Value')
wide_im_ex2021 = im_exports2021_wide
wide_im_ex2021

I'm gonna try with ploty https://plotly.com/python/bar-charts/

In [ ]:
#I tried with plotly but it did not work... still, I attach here the code but "commented out".
#wide_im_ex2021

#import plotly.express as px
#df = px.wide_im_ex2021()
#fig = px.bar(df, x="country", y="Trend_Indicator_Value",color='type_of_exchange', barmode='group',height=400)
#fig.show()
#I don't know why this does not work

I am going to visualize the balance in an interactive bar graph using pygal. ATTENTION: the following bar chart only represents the difference between imports and exports of the countries that had both data of importing and exporting. This is why **in the end I decided not to feature it in my story; I would need to complete it with more data and countries**.

In [ ]:
bar_chart_TIVbalance = pygal.Bar(height=600, margin=5, legend_at_bottom=True, legend_at_bottom_columns=4, legend_box_size=9)
bar_chart_TIVbalance.title = 'Balance Arms Imports - Exports SIPRI TIV'
bar_chart_TIVbalance.add('India', 4381.0)
bar_chart_TIVbalance.add('Qatar', 1738.0)
bar_chart_TIVbalance.add('Australia', 1062.0)
bar_chart_TIVbalance.add('Pakistan', 831.0)
bar_chart_TIVbalance.add('Norway', 434.0)
bar_chart_TIVbalance.add('UAE', 392.0)
bar_chart_TIVbalance.add('Denmark', 318)
bar_chart_TIVbalance.add('Indonesia', 311)
bar_chart_TIVbalance.add('UK', 292.0)
bar_chart_TIVbalance.add('Netherlands', 290.0)
bar_chart_TIVbalance.add('Nigeria', 260.0)
bar_chart_TIVbalance.add('Belgium', 207.0)
bar_chart_TIVbalance.add('South Korea', 154.0)
bar_chart_TIVbalance.add('Brasil', 146.0)
bar_chart_TIVbalance.add('Taiwan', 135.0)
bar_chart_TIVbalance.add('Canada', 135.0)
bar_chart_TIVbalance.add('Thailand', 133.0)
bar_chart_TIVbalance.add('Lithuania', 125.0)
bar_chart_TIVbalance.add('Poland', 106.0)
bar_chart_TIVbalance.add('Serbia', 99.0)
bar_chart_TIVbalance.add('Jordan', 52.0)
bar_chart_TIVbalance.add('Czechia', 29.0)
bar_chart_TIVbalance.add('Finland', 22.0)
bar_chart_TIVbalance.add('Austria', 18.0)
bar_chart_TIVbalance.add('Israel', 4.0)
bar_chart_TIVbalance.add('Bulgaria', -2.0)
bar_chart_TIVbalance.add('Portugal', -4.0)
bar_chart_TIVbalance.add('Belarus', -36.0)
bar_chart_TIVbalance.add('Ukraine', -47.0)
bar_chart_TIVbalance.add('Turkey', -109.0)
bar_chart_TIVbalance.add('Switzerland', -137.0)
bar_chart_TIVbalance.add('China', -184.0)
bar_chart_TIVbalance.add('Sweden', -187.0)
bar_chart_TIVbalance.add('Spain', -487.0)
bar_chart_TIVbalance.add('Germany', -755.0)
bar_chart_TIVbalance.add('Italy', -1488.0)
bar_chart_TIVbalance.add('France', -3908.0)
bar_chart_TIVbalance.add('US', -9875.0)
bar_chart_TIVbalance.render_to_file('bar_chart_TIVbalance.svg') 

In [ ]:
bar_chart_TIVbalance.render_in_browser()